# STEP #1: IMPORT LIBRARIES/DATASETS

In [ ]:
# Load Dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from tensorflow.keras.optimizers import RMSprop
import numpy as np

In [ ]:
# Check Dataset Shapes

from keras.datasets import cifar10
(X_train, y_train) , (X_test, y_test) = cifar10.load_data()

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

# STEP #2: VISUALIZE DATA

In [ ]:
# Visualize an image from the dataset
i = 30009
plt.imshow(X_train[i])
print(y_train[i])

In [ ]:
# Randomly visualizing training images for a sanity check


W_grid = 4
L_grid = 5

fig, axes = plt.subplots(L_grid, W_grid, figsize = (12, 12))
axes = axes.ravel()

n_training = len(X_train)

for i in np.arange(0, L_grid * W_grid):
    index = np.random.randint(0, n_training) # pick a random number
    axes[i].imshow(X_train[index])
    axes[i].set_title(y_train[index])
    axes[i].axis('off')

plt.subplots_adjust(hspace = 0.7)

In [ ]:
n_training

# STEP #3: DATA PREPARATION

In [ ]:
# Converts the image data from integers (0–255) to floating-point numbers (float32). (Neural networks work better with float data)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
number_cat = 10

In [ ]:
y_train

In [ ]:
import keras

# converts class labels (integers) into one-hot encoded vectors
y_train = keras.utils.to_categorical(y_train, number_cat)

In [ ]:
y_train

In [ ]:
y_test = keras.utils.to_categorical(y_test, number_cat)

In [ ]:
y_test

In [ ]:
# normalizes the image pixel values from the range [0, 255] to [0, 1]

X_train = X_train/255
X_test = X_test/255


In [ ]:
X_train

In [ ]:
X_train.shape

In [ ]:
Input_shape = X_train.shape[1:]

In [ ]:
Input_shape

# STEP #4: TRAIN THE MODEL

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

In [ ]:
# stack of layers
cnn_model = Sequential()

# 1st convolutional laye
cnn_model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', input_shape = Input_shape))
#2 nd convolutional laye
cnn_model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'))

# reduces the size of the feature map by picking the maximum value in each 2x2 block.
cnn_model.add(MaxPooling2D(2,2))

#  randomly disables a 40% portion of neurons during training
cnn_model.add(Dropout(0.4))


cnn_model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu'))
cnn_model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(2,2))
cnn_model.add(Dropout(0.4))

# Converts the 2D feature maps into a 1D vector to pass into the dense layers
cnn_model.add(Flatten())

cnn_model.add(Dense(units = 1024, activation = 'relu'))

cnn_model.add(Dense(units = 1024, activation = 'relu'))

cnn_model.add(Dense(units = 10, activation = 'softmax'))

In [ ]:
#  configures CNN model for training
cnn_model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(learning_rate=0.0001), metrics = ['accuracy'])



 # loss='categorical_crossentropy',
 #    optimizer=RMSprop(learning_rate=0.001),
 #    metrics=['accuracy']

In [ ]:
#  trains your CNN model on the training data
history = cnn_model.fit(X_train, y_train, batch_size = 32, epochs = 10, shuffle = True)

# STEP #5: EVALUATE THE MODEL

In [ ]:
evaluation = cnn_model.evaluate(X_test, y_test)
print('Test Accuracy: {}'.format(evaluation[1]))

In [ ]:
# Get class predictions by taking argmax of the predicted probabilities
predicted_classes = np.argmax(cnn_model.predict(X_test), axis=1)

In [ ]:
y_test

In [ ]:
y_test = y_test.argmax(1)

In [ ]:
y_test

In [ ]:
L = 7
W = 7
fig, axes = plt.subplots(L, W, figsize = (12, 12))
axes = axes.ravel()

for i in np.arange(0, L*W):
    axes[i].imshow(X_test[i])
    axes[i].set_title('Prediction = {}\n True = {}'.format(predicted_classes[i], y_test[i]))
    axes[i].axis('off')

plt.subplots_adjust(wspace = 1)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, predicted_classes)
cm
plt.figure(figsize = (10, 10))
sns.heatmap(cm, annot = True)

# STEP #6: SAVING THE MODEL

In [ ]:
import os
directory = os.path.join(os.getcwd(), 'saved_models')

if not os.path.isdir(directory):
    os.makedirs(directory)
model_path = os.path.join(directory, 'keras_cifar10_trained_model.h5')
cnn_model.save(model_path)

# STEP #7: IMPROVING THE MODEL WITH DATA AUGMENTATION

- Image Augmentation is the process of artificially increasing the variations of the images in the datasets by flipping, enlarging, rotating the original images.
- Augmentations also include shifting and changing the brightness of the images.

# STEP 7.1 DATA AUGMENTATION FOR THE CIFAR-10 DATASET

In [ ]:
import keras
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
X_train.shape

In [ ]:
n = 8
X_train_sample = X_train[:n]

In [ ]:
X_train_sample.shape

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# dataget_train = ImageDataGenerator(rotation_range = 90)
# dataget_train = ImageDataGenerator(vertical_flip=True)
# dataget_train = ImageDataGenerator(height_shift_range=0.5)
dataget_train = ImageDataGenerator(brightness_range=(1,3))


dataget_train.fit(X_train_sample)

In [ ]:
from PIL import Image


fig = plt.figure(figsize=(20, 2))

for x_batch in dataget_train.flow(X_train_sample, batch_size=n):
    for i in range(0, n):
        ax = fig.add_subplot(1, n, i + 1)

        # Ensure the image is in the correct format (uint8)
        img = Image.fromarray(x_batch[i].astype('uint8'))
        ax.imshow(img)

    fig.suptitle('Augmented images (rotated 90 degrees)')
    plt.show()
    break

# STEP 7.2 MODEL TRAINING USING AUGEMENTED DATASET

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(
                            rotation_range = 90,
                            width_shift_range = 0.1,
                            horizontal_flip = True,
                            vertical_flip = True
                             )

In [ ]:
datagen.fit(X_train)

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_cat = to_categorical(y_train, num_classes=10)

cnn_model.fit(datagen.flow(X_train, y_train_cat, batch_size=32), epochs=2)


In [ ]:
from keras.utils import to_categorical
from keras.datasets import cifar10

# Load CIFAR-10 test set
(_, _), (X_test, y_test) = cifar10.load_data()

# Flatten and one-hot encode labels
y_test = y_test.flatten()
y_test = to_categorical(y_test, num_classes=10)

# Evaluate the model
score = cnn_model.evaluate(X_test, y_test)
print('Test accuracy:', score[1])


In [ ]:
# save the model
directory = os.path.join(os.getcwd(), 'saved_models')

if not os.path.isdir(directory):
    os.makedirs(directory)
model_path = os.path.join(directory, 'keras_cifar10_trained_model_Augmentation.h5')
cnn_model.save(model_path)

# GREAT JOB!